In [4]:
import requests

url_ZH = 'http://api.openweathermap.org/data/2.5/forecast?lat=47.3744489&lon=8.5410422&units=metric&appid=127619cee40c71033954846a9bfcf283'
api_response = requests.get(url_ZH)
Zurich = api_response.json()
    
main_temps = []
main_temp_min = []
main_temp_max = []
main_humidity = []
weather_main = []
weather_description = []
clouds = []
wind = []
visibility = []
precipitation = []
rain = []
snow = []
datetime = []
city = Zurich.get('city', {}).get('name')
lon = Zurich.get('city', {}).get('coord', {}).get('lon')
lat = Zurich.get('city', {}).get('coord', {}).get('lat')

for i, entry in enumerate(Zurich['list']):
    if i % 8 == 0:  # select 1 value every 24 hours
        main_temps.append(entry['main'].get('temp'))
        main_temp_min.append(entry['main'].get('temp_min'))
        main_temp_max.append(entry['main'].get('temp_max'))
        main_humidity.append(entry['main'].get('humidity'))
        for weather_condition in entry['weather']:
            weather_main.append(weather_condition.get('main'))
            weather_description.append(weather_condition.get('description'))
        clouds.append(entry.get('clouds', {}).get('all'))
        wind.append(entry.get('wind', {}).get('speed'))
        visibility.append(entry.get('visibility'))
        precipitation.append(entry.get('pop'))
        rain.append(entry.get('rain', {}).get('3h'))
        snow.append(entry.get('snow', {}).get('3h'))
        datetime.append(entry.get('dt_txt'))

In [5]:
print(main_temps)
print(main_temp_min)
print(main_temp_max)
print(main_humidity)
print(weather_main)
print(weather_description)
print(clouds)
print(wind)
print(visibility)
print(precipitation)
print(rain)
print(snow)
print(datetime)
print(city)
print(lon)
print(lat)

[6.65, 6.63, 10.25, 9.63, 5.99]
[6.65, 6.63, 10.25, 9.63, 5.99]
[9.62, 6.63, 10.25, 9.63, 5.99]
[92, 72, 95, 93, 89]
['Rain', 'Clouds', 'Rain', 'Rain', 'Rain']
['moderate rain', 'overcast clouds', 'moderate rain', 'moderate rain', 'light rain']
[100, 100, 100, 100, 97]
[6.74, 3.56, 2.44, 3.33, 2.98]
[10000, 10000, 4646, 4782, 10000]
[1, 0.06, 1, 1, 0.84]
[9.33, None, 8.16, 8.46, 0.12]
[None, None, None, None, None]
['2023-12-09 21:00:00', '2023-12-10 21:00:00', '2023-12-11 21:00:00', '2023-12-12 21:00:00', '2023-12-13 21:00:00']
Zürich (Kreis 1) / Lindenhof
8.541
47.3744


In [13]:
import sqlite3

# Connect to the SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('weather_data.db')
cursor = conn.cursor()

# Create a table to store weather data
cursor.execute('''
    CREATE TABLE IF NOT EXISTS weather_data (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        main_temp REAL,
        temp_min REAL,
        temp_max REAL,
        humidity INTEGER,
        weather_main TEXT,
        weather_description TEXT,
        clouds INTEGER,
        wind_speed REAL,
        visibility INTEGER,
        precipitation REAL,
        rain_3h REAL,
        snow_3h REAL,
        datetime TEXT,
        city TEXT,
        lon REAL,
        lat REAL
    )
''')


In [14]:
# Insert data into the table
for i in range(len(main_temps)):
    cursor.execute('''
        INSERT INTO weather_data (
            main_temp, temp_min, temp_max, humidity, weather_main, weather_description,
            clouds, wind_speed, visibility, precipitation, rain_3h, snow_3h, datetime,
            city, lon, lat
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (
        main_temps[i], main_temp_min[i], main_temp_max[i], main_humidity[i],
        weather_main[i], weather_description[i], clouds[i], wind[i],
        visibility[i], precipitation[i], rain[i], snow[i], datetime[i],
        city, lon, lat
    ))

# Commit the changes and close the connection
conn.commit()
conn.close()

In [17]:
# Connect to the database
conn = sqlite3.connect('weather_data.db')
cursor = conn.cursor()

# Execute a SELECT query to retrieve data from the table
cursor.execute('SELECT * FROM weather_data')

# Fetch all rows from the result set
rows = cursor.fetchall()

# Print the table header
header = [description[0] for description in cursor.description]
print("\t".join(map(str, header)))

# Print each row of the table
for row in rows:
    print("\t".join(map(str, row)))

# Close the connection
conn.close()


id	main_temp	temp_min	temp_max	humidity	weather_main	weather_description	clouds	wind_speed	visibility	precipitation	rain_3h	snow_3h	datetime	city	lon	lat
1	6.65	6.65	9.62	92	Rain	moderate rain	100	6.74	10000	1.0	9.33	None	2023-12-09 21:00:00	Zürich (Kreis 1) / Lindenhof	8.541	47.3744
2	6.63	6.63	6.63	72	Clouds	overcast clouds	100	3.56	10000	0.06	None	None	2023-12-10 21:00:00	Zürich (Kreis 1) / Lindenhof	8.541	47.3744
3	10.25	10.25	10.25	95	Rain	moderate rain	100	2.44	4646	1.0	8.16	None	2023-12-11 21:00:00	Zürich (Kreis 1) / Lindenhof	8.541	47.3744
4	9.63	9.63	9.63	93	Rain	moderate rain	100	3.33	4782	1.0	8.46	None	2023-12-12 21:00:00	Zürich (Kreis 1) / Lindenhof	8.541	47.3744
5	5.99	5.99	5.99	89	Rain	light rain	97	2.98	10000	0.84	0.12	None	2023-12-13 21:00:00	Zürich (Kreis 1) / Lindenhof	8.541	47.3744
6	6.65	6.65	9.62	92	Rain	moderate rain	100	6.74	10000	1.0	9.33	None	2023-12-09 21:00:00	Zürich (Kreis 1) / Lindenhof	8.541	47.3744
7	6.63	6.63	6.63	72	Clouds	overcast clouds	100	3.56	10000

In [16]:
# Connect to the database
conn = sqlite3.connect('weather_data.db')
cursor = conn.cursor()

# Check if the table exists
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='weather_data'")
table_exists = cursor.fetchone() is not None

# Print the result
if table_exists:
    print("The 'weather_data' table exists.")
else:
    print("The 'weather_data' table does not exist.")

# Close the connection
conn.close()


The 'weather_data' table exists.
